In [100]:
!nvidia-smi

Mon Nov 20 17:50:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060      WDDM  | 00000000:01:00.0  On |                  N/A |
| 44%   28C    P8              12W / 184W |    708MiB / 12288MiB |     13%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [101]:
import numpy as np 
import random
from scipy.optimize import minimize
from sphere_decoding.sphereDecodingUseC import sphere_decoding_BER
from numba import cuda, jit

In [111]:
@jit(nopython=True)
def generate_random_bit_sequence(length):
    x = np.ones(length)
    for ii in range(length):
        x[ii] = round(np.random.random())
    return x

%timeit generate_random_bit_sequence(1024)


4.94 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [128]:
@jit(nopython=True)
def qam16_modulation(binary_input):
    x = np.array([8,4,2,1],dtype=np.float32)
    index =  np.dot(binary_input, x)   
    mapping = np.array([(1+1j),(1+3j),(3+1j),(3+3j),(1-1j),(1-3j),(3-1j),(3-3j),(-1+1j),(-1+3j),(-3+1j),(-3+3j),(-1-1j),(-1-3j),(-3-1j),(-3-3j)])
    return mapping[np.round(index)]/np.sqrt(10)

qam16_modulation(np.array([1,1,1,1],dtype=np.float32)) 

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(complex128, 1d, C), float64)
 
There are 22 candidate implementations:
[1m      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(array(complex128, 1d, C), float64)':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba\core\typing\arraydecl.py: Line 209.
        With argument(s): '(array(complex128, 1d, C), float64)':[0m
[1m       Rejected as the implementation raised a specific error:
         NumbaTypeError: [1mUnsupported array index type float64 in [float64][0m[0m
  raised from d:\Software\Anaconda\envs\qnn\Lib\site-packages\numba\core\typing\arraydecl.py:102
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\WenKai\App

In [115]:
@jit(nopython=True)
def generate_x_sequence(length, Nt):
    total_bits_sequence = generate_random_bit_sequence(length*Nt*4)
    x_sequence = np.zeros(length*Nt).astype(np.complex128)
    for ii in range(length*Nt):
        x_sequence[ii] = qam16_modulation(total_bits_sequence[4*ii:4*ii+4])
    return x_sequence
generate_x_sequence(10, 2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'qam16_modulation':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "..\..\..\AppData\Local\Temp\ipykernel_12612\181705542.py", line 6:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [183]:
H = np.random.randn(4,2)+1j*np.random.randn(4,2)
y = np.array([1,1,1,1],dtype=np.complex128).reshape(4,1)
Nt = 2
Nr = 4
def bits2signals(bits):
    # bits: input binary string with length of (4*Nt) 
    return np.array([qam16_modulation(bits[i:i+4]) for i in range(0, len(bits), 4)]).reshape(Nt,1)
def qam16_modulation(binary_input):
    mapping = {
        '0000': (1+1j),
        '0001': (1+3j),
        '0010': (3+1j),
        '0011': (3+3j),
        '0100': (1-1j),
        '0101': (1-3j),
        '0110': (3-1j),
        '0111': (3-3j),
        '1000': (-1+1j),
        '1001': (-1+3j),
        '1010': (-3+1j),
        '1011': (-3+3j),
        '1100': (-1-1j),
        '1101': (-1-3j),
        '1110': (-3-1j),
        '1111': (-3-3j)
    }
    return mapping.get(binary_input, "Invalid binary input")/np.sqrt(10)


def calculate_layer1(H_hat, y):
    dimension_layer1 = 2**(4*Nt)
    output = {}
    for index in range(dimension_layer1):
        bits = str(bin(index)[2:].zfill(4*Nt))
        s = bits2signals(bits)
        error = y - np.dot(H_hat,s)
        value =  np.exp(-np.square(np.linalg.norm(error)))
        output[bits] = value
    return output




%time output1 = calculate_layer1(H, y)



CPU times: total: 0 ns
Wall time: 5 ms


In [187]:
from numba import cuda
import math
from timeit import default_timer as time

# @cuda.jit 
# def calculate_layer1_GPU(H_hat, y, output_layer1, length):
#     pos = cuda.grid(1)
#     if pos < length:
#         output_layer1[pos] = pos


# total_len = 2**(H.shape[1]*4)
# threadsperblock = 128
# blockspergrid = math.ceil(total_len/threadsperblock)
# output1 = np.zeros(total_len)

# # s = time()
# %time calculate_layer1_GPU[blockspergrid, threadsperblock](H, y, output1, total_len)
# # print(time()-s)

@cuda.jit 
def qam16_modulation(binary_input):
    x = np.array([8,4,2,1],dtype=np.float32)
    index =  np.dot(binary_input, x)   
    mapping = np.array([(1+1j),(1+3j),(3+1j),(3+3j),(1-1j),(1-3j),(3-1j),(3-3j),(-1+1j),(-1+3j),(-3+1j),(-3+3j),(-1-1j),(-1-3j),(-3-1j),(-3-3j)])
    return mapping[np.round(index)]/np.sqrt(10)

qam16_modulation[1,1](np.array([1,1,1,1],dtype=np.float32)) 

C:\Users\WenKai\AppData\Local\Temp\ipykernel_12612\3858807461.py:23: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-captured-errors
Exception origin:
  File "d:\Software\Anaconda\envs\qnn\Lib\site-packages\numba\core\untyped_passes.py", line 125, in run_pass
    raise TypeError("Signature mismatch: %d argument types given, "

  x = np.array([8,4,2,1],dtype=np.float32)
C:\Users\WenKai\AppData\Local\Temp\ipykernel_12612\3858807461.py:25: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-capture

TypingError: Failed in cuda mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(int64)<iv=[8, 4, 2, 1]>, dtype=class(float32))
 
There are 2 candidate implementations:
[1m   - Of which 2 did not match due to:
   Overload in function 'impl_np_array': File: numba\np\arrayobj.py: Line 5384.
     With argument(s): '(list(int64)<iv=None>, dtype=class(float32))':[0m
[1m    Rejected as the implementation raised a specific error:
      TypeError: Failed in cuda mode pipeline (step: fix up args)
    Signature mismatch: 1 argument types given, but function takes 2 arguments[0m
  raised from d:\Software\Anaconda\envs\qnn\Lib\site-packages\numba\core\untyped_passes.py:125
[0m
[0m[1mDuring: resolving callee type: Function(<built-in function array>)[0m
[0m[1mDuring: typing of call at C:\Users\WenKai\AppData\Local\Temp\ipykernel_12612\3858807461.py (23)
[0m
[1m
File "..\..\..\AppData\L

In [167]:
from numba import jit, guvectorize, int32, int64, float64
from numba import cuda
import numpy as np
import math

@cuda.jit (device=True)
def sub_stuff(vec_a, vec_b, result):
    for i in range(vec_a.shape[0]):
        result[i] = vec_a[i] - vec_b[i]

@cuda.jit
def kernel_via_polygon(vectors_a, vectors_b, result_array):
    pos = cuda.grid(1)
    if pos < vectors_a.size and pos < result_array.size:
         sub_stuff(vectors_a[pos], vectors_b[pos], result_array[pos])

vectors_a = 100 + np.arange(1, 10).reshape((3, 3))
vectors_b = np.arange(1, 10).reshape((3, 3))
result = np.zeros_like(vectors_a)

kernel_via_polygon[1,10](vectors_a, vectors_b, result)

print(result)

[[100 100 100]
 [100 100 100]
 [100 100 100]]


d:\Software\Anaconda\envs\qnn\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\Software\Anaconda\envs\qnn\Lib\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [164]:
H.shape

(4, 2)